In [1]:
import os
from os import listdir
from os import path
from os.path import isfile, join, getsize
import re
import pandas as pd
import random

import json
import io
import glob
import pickle

#need this for some stupid reason to import essentia
import sys
sys.path.append("/usr/local/lib/python3.6/dist-packages")
import essentia
import essentia.standard as es
import multiprocessing as mp

from essentia.pytools.spectral import hpcpgram
%matplotlib inline
import matplotlib.pyplot as plt

from pexecute.process import ProcessLoom
loom = ProcessLoom(max_runner_cap=30)


Bad key "text.kerning_factor" on line 4 in
/opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:

ext_drive_dir = '/home/jupyter-chandler.vaughn/'
working_root_dir = 'covers32k/'
output_dir = 'audio_output_stems/'
#png_folder = 'png_output/'
#harmonic_pitch_class_profile = 'harmonic_pitch_class_profile/'
#cover_song_simularity = 'cover_song_simularity/'
#cross_recurrent_plot = 'cross_recurrent_plot/'

#audio_stems = 'audio_output_stems/'

os.chdir(ext_drive_dir+working_root_dir)


#get all the directories for the songs
directories_songs = []
for x in os.walk(ext_drive_dir+working_root_dir):
    directories_songs.append(x[1])

directories_songs = directories_songs[0]
directories_songs.remove('audio_output_stems')
directories_songs.remove('pretrained_models')
directories_songs[:5]

['Grand_Illusion',
 'Summer_of_69',
 'Train_In_Vain',
 'Yesterday',
 'Blue_Collar_Man']

In [3]:
#get all the directories for the stems
directories_stems = []
for x in os.walk(ext_drive_dir+working_root_dir+output_dir):
    directories_stems.append(x[1])

directories_stems = directories_stems[0]
directories_stems[:5]

['dave_matthews_band+Live_At_Red_Rocks_8_15_95_Disc_2+08-All_Along_The_Watchtower',
 'celine_dion+Falling_Into_You+12-River_Deep_Mountain_High',
 'tina_turner+Tina_Live_In_Europe_CD_2_+07-Proud_Mary',
 'boomtown_rats+Best_of_the_Boomtown_Rats+10-I_Don_t_Like_Mondays',
 'styx+Caught_In_The_Act_Disc_2_of_2_+02-Blue_Collar_Man_Long_Nights_']

In [4]:
filename_list1 = []
filestem_list1 = []

global processed
global result
processed = []
result_template = pd.DataFrame({'pair':[], 
                       'song_1':[],
                       'song_2':[],
                       'MFCC_1_bass':[], 
                       'MFCC_1_drums':[],
                       'MFCC_1_vocals':[],
                       'MFCC_1_other':[],
                       'GFCC_1_bass':[],
                       'GFCC_1_drums':[],
                       'GFCC_1_vocals':[],
                       'GFCC_1_other':[],
                       'HPCP_1_bass':[],
                       'HPCP_1_drums':[],
                       'HPCP_1_vocals':[],
                       'HPCP_1_other':[],
                       'MFCC_2_bass':[], 
                       'MFCC_2_drums':[],
                       'MFCC_2_vocals':[],
                       'MFCC_2_other':[],
                       'GFCC_2_bass':[],
                       'GFCC_2_drums':[],
                       'GFCC_2_vocals':[],
                       'GFCC_2_other':[],
                       'HPCP_2_bass':[],
                       'HPCP_2_drums':[],
                       'HPCP_2_vocals':[],
                       'HPCP_2_other':[],
                       'sim_matrix_bass':[],
                       'sim_matrix_drums':[],
                       'sim_matrix_vocals':[],
                       'sim_matrix_other':[],
                       'sim_distance_bass':[],
                       'sim_distance_drums':[],
                       'sim_distance_vocals':[],
                       'sim_distance_other':[],
                       'sim_distance_argmin':[],
                       'MFCC_1':[], 
                       'GFCC_1':[],
                       'HPCP_1':[],
                       'MFCC_2':[], 
                       'GFCC_2':[],
                       'HPCP_2':[],
                       'sim_matrix':[],
                       'sim_distance':[]})

result = pd.DataFrame({'pair':[], 
                       'song_1':[],
                       'song_2':[],
                       'MFCC_1_bass':[], 
                       'MFCC_1_drums':[],
                       'MFCC_1_vocals':[],
                       'MFCC_1_other':[],
                       'GFCC_1_bass':[],
                       'GFCC_1_drums':[],
                       'GFCC_1_vocals':[],
                       'GFCC_1_other':[],
                       'HPCP_1_bass':[],
                       'HPCP_1_drums':[],
                       'HPCP_1_vocals':[],
                       'HPCP_1_other':[],
                       'MFCC_2_bass':[], 
                       'MFCC_2_drums':[],
                       'MFCC_2_vocals':[],
                       'MFCC_2_other':[],
                       'GFCC_2_bass':[],
                       'GFCC_2_drums':[],
                       'GFCC_2_vocals':[],
                       'GFCC_2_other':[],
                       'HPCP_2_bass':[],
                       'HPCP_2_drums':[],
                       'HPCP_2_vocals':[],
                       'HPCP_2_other':[],
                       'sim_matrix_bass':[],
                       'sim_matrix_drums':[],
                       'sim_matrix_vocals':[],
                       'sim_matrix_other':[],
                       'sim_distance_bass':[],
                       'sim_distance_drums':[],
                       'sim_distance_vocals':[],
                       'sim_distance_other':[],
                       'sim_distance_argmin':[],
                       'MFCC_1':[], 
                       'GFCC_1':[],
                       'HPCP_1':[],
                       'MFCC_2':[], 
                       'GFCC_2':[],
                       'HPCP_2':[],
                       'sim_matrix':[],
                       'sim_distance':[]})

#set file names since they are all the same
bass = 'bass.wav'
drums = 'drums.wav'
voice = 'vocals.wav'
other = 'other.wav'

stems = (bass, drums, voice, other)

#look for mp3 and wav.
#the wav file here is the baseline... a guassian whitenoise at 0db

types = ('*.mp3', '*.wav')
#find all file pairs
for f in types:
    filename_list1.extend(glob.glob(f))
    
#split the filenames
for pair in directories_songs:
    for d in directories_stems:
        if path.exists(ext_drive_dir+working_root_dir+pair+'/'+d+'.mp3') == True:
            os.chdir(ext_drive_dir+working_root_dir+output_dir+d)
            for f in glob.glob('*.wav'):
                filestem = f.split('.')
                filestem_list1.append([pair,d,filestem[0],filestem[1]])

filestem_list2 = filestem_list1
filestem_list1[:5]

[['Grand_Illusion',
  'reo_speedwagon+Arch_Allies_Disc_1+02-Grand_Illusion',
  'vocals',
  'wav'],
 ['Grand_Illusion',
  'reo_speedwagon+Arch_Allies_Disc_1+02-Grand_Illusion',
  'drums',
  'wav'],
 ['Grand_Illusion',
  'reo_speedwagon+Arch_Allies_Disc_1+02-Grand_Illusion',
  'other',
  'wav'],
 ['Grand_Illusion',
  'reo_speedwagon+Arch_Allies_Disc_1+02-Grand_Illusion',
  'bass',
  'wav'],
 ['Grand_Illusion',
  'styx+Return_To_Paradise_Disc_2+01-Grand_Illusion',
  'vocals',
  'wav']]

In [5]:
from essentia.standard import *

logNorm = UnaryOperator(type='log')

def extract_mfcc(audio_location):
    audio = es.MonoLoader(filename=audio_location, sampleRate=32000)()
    w = Windowing(type = 'hann')
    spectrum = Spectrum()
    mfcc = essentia.standard.MFCC()
    mfccs =[]
    melbands = []
    audio = essentia.array(audio)
    for frame in FrameGenerator(audio, frameSize = 1024 , hopSize = 512):
        mfcc_bands, mfcc_coeffs = mfcc(spectrum(w(frame)))
        mfccs.append(mfcc_coeffs)
        melbands.append(mfcc_bands)
    mfcc = essentia.array(melbands).T
    return mfcc

def extract_gfcc(audio_location):
    audio = es.MonoLoader(filename=audio_location, sampleRate=32000)()
    w = Windowing(type = 'hann')
    spectrum = Spectrum()
    gfcc = essentia.standard.GFCC()
    gfccs =[]
    gfccbands = []
    audio = essentia.array(audio)
    for frame in FrameGenerator(audio, frameSize = 1024 , hopSize = 512):
        gfcc_bands, gfcc_coeffs = gfcc(spectrum(w(frame)))
        gfccs.append(gfcc_coeffs)
        gfccbands.append(gfcc_bands)
    gfcc = essentia.array(gfccbands).T
    return gfcc

def extract_hpcp(audio_location):
    audio = es.MonoLoader(filename=audio_location, sampleRate=32000)()
    hpcp = hpcpgram(audio, sampleRate=32000)
    #hpcp = essentia.array(logNorm(hpcp)).T
    return hpcp

def extract_cover_similarity(f1_location, f2_location):
    #Calculate ChromaCrossSimularity
    song_1 = es.MonoLoader(filename=f1_location, sampleRate=32000)()
    song_2 = es.MonoLoader(filename=f2_location, sampleRate=32000)()
    
    
    #loom.add_function(extract_hpcp,[f1_location],{},'song_1_hpcp')
    #loom.add_function(extract_hpcp,[f2_location],{},'song_2_hpcp')
    #output_hpcp = loom.execute()
    #print(output_hpcp)
    #song_1_hpcp = output_hpcp['song_1_hpcp']['output']
    #song_2_hpcp = output_hpcp['song_2_hpcp']['output']
    
    song_1_hpcp = extract_hpcp(f1_location)
    song_2_hpcp = extract_hpcp(f2_location)
    crp = es.ChromaCrossSimilarity(frameStackSize=9, 
                                 frameStackStride=1, 
                                 binarizePercentile=0.095,
                                 oti=True)

    true_pair_crp = crp(song_1_hpcp, song_2_hpcp)
    
    #Calculate Distance with CSS
    score_matrix, distance = es.CoverSongSimilarity(disOnset=0.5, 
                                                  disExtension=0.5, 
                                                  alignmentType='chen17',
                                                  distanceType='asymmetric')(true_pair_crp)
    
    return score_matrix, distance, song_1_hpcp, song_2_hpcp

In [6]:
def process_file(file1, file2, directory1, directory2, q):
    global result
    global processed
    argmin = 0
    distance_bass = 0
    distance_drums = 0
    distance_voice = 0
    distance_other = 0

    if file1 not in processed:
        #print(directory1)
        f2 = file2
        dir2 = directory2

        full_1 = ext_drive_dir+working_root_dir+directory1+'/'+file1+'.mp3'
        full_2 = ext_drive_dir+working_root_dir+dir2+'/'+f2+'.mp3'
        #print("*** Full File Processing ***")
        print(full_1)
        print(full_2)

        loom.add_function(extract_mfcc,[full_1],{},'mfcc_1')
        loom.add_function(extract_mfcc,[full_2],{},'mfcc_2')
        loom.add_function(extract_gfcc,[full_1],{},'gfcc_1')
        loom.add_function(extract_gfcc,[full_2],{},'gfcc_1')
        output_full = loom.execute()
        #print(full_1)
        #print(output_full)
        mfcc_1 = output_full['mfcc_1']['output']
        mfcc_2 = output_full['mfcc_2']['output']
        gfcc_1 = output_full['gfcc_1']['output']
        gfcc_2 = output_full['gfcc_1']['output']
        
        #print(mfcc_1)
        #print(mfcc_2)
        #print(gfcc_1)
        #print(gfcc_2)
        #mfcc_1 = extract_mfcc(full_1)
        #mfcc_2 = extract_mfcc(full_2)

        #gfcc_1 = extract_gfcc(full_1)
        #gfcc_2 = extract_gfcc(full_2)

        #calculate HPCP
        #hpcp_1 = extract_hpcp(filename1)
        #hpcp_2 = extract_hpcp(filename2)

        #calculate similarity score
        score_matrix_full, distance_full, song_1_hpcp_full, song_2_hpcp_full = extract_cover_similarity(full_1, full_2)

        for f in stems:
        #f = i[2]+'.'+i[3]
            filename1 = ext_drive_dir+working_root_dir+output_dir+file1 + '/' + f 
            filename2 = ext_drive_dir+working_root_dir+output_dir+f2 + '/' + f
            print(filename1)
            print(filename2)
            if f == 'bass.wav':
                loom.add_function(extract_mfcc,[filename1],{},'mfcc_1_bass')
                loom.add_function(extract_mfcc,[filename2],{},'mfcc_2_bass')
                loom.add_function(extract_gfcc,[filename1],{},'gfcc_1_bass')
                loom.add_function(extract_gfcc,[filename2],{},'gfcc_2_bass')
                output_bass = loom.execute()
                
                mfcc_1_bass = output_bass['mfcc_1_bass']['output']
                mfcc_2_bass = output_bass['mfcc_2_bass']['output']
                gfcc_1_bass = output_bass['gfcc_1_bass']['output']
                gfcc_2_bass = output_bass['gfcc_2_bass']['output']
                
                #mfcc_1_bass = extract_mfcc(filename1)
                #mfcc_2_bass = extract_mfcc(filename2)

                #gfcc_1_bass = extract_gfcc(filename1)
                #gfcc_2_bass = extract_gfcc(filename2)

                score_matrix_bass, distance_bass, song_1_hpcp_bass, song_2_hpcp_bass = extract_cover_similarity(filename1, filename2)

            elif f == 'drums.wav':
                loom.add_function(extract_mfcc,[filename1],{},'mfcc_1_drums')
                loom.add_function(extract_mfcc,[filename2],{},'mfcc_2_drums')
                loom.add_function(extract_gfcc,[filename1],{},'gfcc_1_drums')
                loom.add_function(extract_gfcc,[filename2],{},'gfcc_2_drums')
                output_drums = loom.execute()
                
                mfcc_1_drums = output_drums['mfcc_1_drums']['output']
                mfcc_2_drums = output_drums['mfcc_2_drums']['output']
                gfcc_1_drums = output_drums['gfcc_1_drums']['output']
                gfcc_2_drums = output_drums['gfcc_2_drums']['output']
                
                #mfcc_1_drums = extract_mfcc(filename1)
                #mfcc_2_drums = extract_mfcc(filename2)

                #gfcc_1_drums = extract_gfcc(filename1)
                #gfcc_2_drums = extract_gfcc(filename2)

                score_matrix_drums, distance_drums, song_1_hpcp_drums, song_2_hpcp_drums = extract_cover_similarity(filename1, filename2)

            elif f == 'vocals.wav':
                loom.add_function(extract_mfcc,[filename1],{},'mfcc_1_vocals')
                loom.add_function(extract_mfcc,[filename2],{},'mfcc_2_vocals')
                loom.add_function(extract_gfcc,[filename1],{},'gfcc_1_vocals')
                loom.add_function(extract_gfcc,[filename2],{},'gfcc_2_vocals')
                output_vocals = loom.execute()
                
                mfcc_1_vocals = output_vocals['mfcc_1_vocals']['output']
                mfcc_2_vocals = output_vocals['mfcc_2_vocals']['output']
                gfcc_1_vocals = output_vocals['gfcc_1_vocals']['output']
                gfcc_2_vocals = output_vocals['gfcc_2_vocals']['output']
                
                #mfcc_1_vocals = extract_mfcc(filename1)
                #mfcc_2_vocals = extract_mfcc(filename2)

                #gfcc_1_vocals = extract_gfcc(filename1)
                #gfcc_2_vocals = extract_gfcc(filename2)

                score_matrix_vocals, distance_vocals, song_1_hpcp_vocals, song_2_hpcp_vocals = extract_cover_similarity(filename1, filename2)

            else:
                loom.add_function(extract_mfcc,[filename1],{},'mfcc_1_other')
                loom.add_function(extract_mfcc,[filename2],{},'mfcc_2_other')
                loom.add_function(extract_gfcc,[filename1],{},'gfcc_1_other')
                loom.add_function(extract_gfcc,[filename2],{},'gfcc_2_other')
                output_other = loom.execute()
                
                mfcc_1_other = output_other['mfcc_1_other']['output']
                mfcc_2_other = output_other['mfcc_2_other']['output']
                gfcc_1_other = output_other['gfcc_1_other']['output']
                gfcc_2_other = output_other['gfcc_2_other']['output']
                
                #mfcc_1_other = extract_mfcc(filename1)
                #mfcc_2_other = extract_mfcc(filename2)

                #gfcc_1_other = extract_gfcc(filename1)
                #gfcc_2_other = extract_gfcc(filename2)

                score_matrix_other, distance_other, song_1_hpcp_other, song_2_hpcp_other = extract_cover_similarity(filename1, filename2)

        



        argset = set([distance_bass,
                   distance_drums,
                   distance_vocals,
                   distance_other])

        argmin = min(argset)



        new_row = {'pair':'random', 
                   'song_1':file1,
                   'song_2':f2,
                   'MFCC_1_bass':mfcc_1_bass, 
                   'MFCC_1_drums':mfcc_1_drums,
                   'MFCC_1_vocals':mfcc_1_vocals,
                   'MFCC_1_other':mfcc_1_other,
                   'GFCC_1_bass':gfcc_1_bass,
                   'GFCC_1_drums':gfcc_1_drums,
                   'GFCC_1_vocals':gfcc_1_vocals,
                   'GFCC_1_other':gfcc_1_other,
                   'HPCP_1_bass':song_1_hpcp_bass,
                   'HPCP_1_drums':song_1_hpcp_drums,
                   'HPCP_1_vocals':song_1_hpcp_vocals,
                   'HPCP_1_other':song_1_hpcp_other,
                   'MFCC_2_bass':mfcc_2_bass, 
                   'MFCC_2_drums':mfcc_2_drums,
                   'MFCC_2_vocals':mfcc_2_vocals,
                   'MFCC_2_other':mfcc_2_other,
                   'GFCC_2_bass':gfcc_2_bass,
                   'GFCC_2_drums':gfcc_2_drums,
                   'GFCC_2_vocals':gfcc_2_vocals,
                   'GFCC_2_other':gfcc_2_other,
                   'HPCP_2_bass':song_2_hpcp_bass,
                   'HPCP_2_drums':song_2_hpcp_drums,
                   'HPCP_2_vocals':song_2_hpcp_vocals,
                   'HPCP_2_other':song_2_hpcp_other,
                   'sim_matrix_bass':score_matrix_bass,
                   'sim_matrix_drums':score_matrix_drums,
                   'sim_matrix_vocals':score_matrix_vocals,
                   'sim_matrix_other':score_matrix_other,
                   'sim_distance_bass':distance_bass,
                   'sim_distance_drums':distance_drums,
                   'sim_distance_vocals':distance_vocals,
                   'sim_distance_other':distance_other,
                   'sim_distance_argmin':argmin,
                   'MFCC_1':mfcc_1, 
                   'GFCC_1':gfcc_1,
                   'HPCP_1':song_1_hpcp_full,
                   'MFCC_2':mfcc_2, 
                   'GFCC_2':gfcc_2,
                   'HPCP_2':song_2_hpcp_full,
                   'sim_matrix':score_matrix_full,
                   'sim_distance':distance_full}

        #print(new_row)
        #df = df.append(new_row, ignore_index=True) 
        result = result.append(new_row, ignore_index=True)  
        q.put(result)
        processed.append(file1)
    return result_template

In [7]:
args = []
preprocess = []
for num1, i in enumerate(filestem_list1):
        if i[1] not in preprocess:
            dir1 = i[0]
            f1 = i[1]
            f2 = i[1]
            while f2 == i[1]:         
                f2 = random.choice(filestem_list1)
                dir2 = f2[0]
                f2 = f2[1]
            
            args.append([f1, f2, dir1, dir2])  
            preprocess.append(f1)

In [8]:
qout = mp.Manager().Queue()
process = [mp.Process(target=process_file, args=(args[ii][0], args[ii][1], 
                                                 args[ii][2], args[ii][3], qout)) for ii in range(len(args))]
for p in process:
    p.start()

for p in process:
    p.join()
    
result = [qout.get() for p in process]

/home/jupyter-chandler.vaughn/covers32k/Grand_Illusion/reo_speedwagon+Arch_Allies_Disc_1+02-Grand_Illusion.mp3
/home/jupyter-chandler.vaughn/covers32k/Grand_Illusion/styx+Return_To_Paradise_Disc_2+01-Grand_Illusion.mp3/home/jupyter-chandler.vaughn/covers32k/Proud_Mary/creedence_clearwater_revival+Live_in_Europe+10-Proud_Mary.mp3
/home/jupyter-chandler.vaughn/covers32k/Summer_of_69/bryan_adams+Live_Live_Live_+13-Summer_of_69.mp3/home/jupyter-chandler.vaughn/covers32k/Downtown_Lights/annie_lennox+Medusa+07-Downtown_Lights.mp3


/home/jupyter-chandler.vaughn/covers32k/Blue_Collar_Man/styx+Caught_In_The_Act_Disc_2_of_2_+02-Blue_Collar_Man_Long_Nights_.mp3/home/jupyter-chandler.vaughn/covers32k/Summer_of_69/mxpx+On_The_Cover+01-Summer_of_69.mp3

/home/jupyter-chandler.vaughn/covers32k/All_Tomorrow_s_Parties/velvet_underground+Andy_Warhol_with_Nico_+06-All_Tomorrow_s_Parties.mp3/home/jupyter-chandler.vaughn/covers32k/Train_In_Vain/clash+London_Calling+19-Train_In_Vain.mp3

/home/jupyter-chan

In [9]:
tmp = pd.DataFrame()
for items in result:
    tmp = tmp.append(items)
    


NameError: name 'test' is not defined

In [10]:
result = tmp
os.chdir('/home/jupyter-chandler.vaughn/data/')

In [29]:


with open('cover32_random.pickle', 'wb') as f:
    pickle.dump(result, f)

In [31]:
result.shape

(164, 44)

In [32]:
infile = open('cover80_song_random_stem_data.pickle','rb')
newdf_from_pickle = pickle.load(infile)
infile.close()

In [33]:
newdf_from_pickle.shape

(164, 44)

In [35]:
newdf_from_pickle.head(2)

,pair,song_1,song_2,MFCC_1_bass,MFCC_1_drums,MFCC_1_vocals,MFCC_1_other,GFCC_1_bass,GFCC_1_drums,GFCC_1_vocals,...,sim_distance_other,sim_distance_argmin,MFCC_1,GFCC_1,HPCP_1,MFCC_2,GFCC_2,HPCP_2,sim_matrix,sim_distance
0,random,everly_brothers+The_Fabulous_Style_of+01-Claud...,queen+Sheer_Heart_Attack+08-Stone_Cold_Crazy,"[[2.731574e-09, 5.028052e-18, 0.0, 0.0, 0.0, 0...","[[5.288408e-09, 3.8340316e-16, 0.0, 0.0, 0.0, ...","[[4.470984e-10, 3.0614603e-17, 0.0, 0.0, 0.0, ...","[[6.722479e-12, 1.525382e-15, 0.0, 0.0, 0.0, 2...","[[2.1485027e-07, 3.9610276e-16, 0.0, 0.0, 0.0,...","[[4.1619845e-07, 3.0247347e-14, 0.0, 0.0, 0.0,...","[[3.513951e-08, 2.4110427e-15, 0.0, 0.0, 0.0, ...",...,0.066534,0.053528,"[[5.2894084e-14, 3.4971077e-14, 7.563816e-14, ...","[[0.0, 0.0, 2.1644388e-12, 3.231479e-11, 5.620...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 2.1644388e-12, 3.231479e-11, 5.620...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.058003
0,random,green_day+Kerplunk+16-My_Generation,big_star+Radio_City+10-September_Gurls,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,0.086629,0.045421,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.79...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.68...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.68...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.062419
